In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-dataset.zip

--2022-08-04 23:54:05--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-dataset.zip [following]
--2022-08-04 23:54:06--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc492224ee82fac8352045d133db.dl.dropboxusercontent.com/cd/0/inline/BqaKHWm_UUp2JfOQdykvuQ0jKDpmuEqmDbAAH4XCMWPJNU_jAE_x1VdE-i2JgDKj2Isq4zJyDm1QBjykMaPTm76vQ7K-pPXdMvmzmGzAFTNDHEEshb19sMudxDHdxfGC0qKqccY5855De-93m8fI8SA-Ha5wcVXhfI3noJfrNVlOCw/file# [following]
--2022-08-04 23:54:06--  https://uc492224ee82fac8352045d133db.dl.dropboxusercontent.com/cd/0/inline/BqaKHWm_UUp2JfOQdykvuQ0jKDpmuEqmDbAAH4XCMWPJNU_jAE_

In [2]:
!unzip -q 'movie-dataset.zip'

In [4]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, SimpleRNNCell, Embedding, Dropout, Bidirectional

In [5]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, '<br />',' '), label),)

In [6]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"I found this film to be the usual French slap in America's face. The camera, all too often, focuses on fat people, on sloppy homes and on tacky rural areas. While the narration seems to sympathize with and admire the small town folks who are introduced to the viewer, the cinematography exploits and demeans them. There were, undoubtedly, thin people to be seen in Glencoe and neat, organized homes, but Malle chose to show us the worst of what was there to be seen.   I can only hope that some American filmmakers will go to France to reveal to the American public its worst elements. I can assure you, as a frequent visitor to France, that all is not well there. Foreign immigrants are not readily assimilated, thus creating severe social inequities. But Americans are not eager to unmask the French for their prejudice toward their own compatriots and their envy toward the U.S., so we're not likely to see film

In [7]:
model = Sequential()
model.add(Input(shape=(1,),dtype='string'))

In [8]:
max_tokens = 1000
max_len = 100

vectorized_layer = TextVectorization(max_tokens = max_tokens,
                                     output_mode = 'int',
                                     output_sequence_length = max_len,)





In [9]:
train_text = train_data.map(lambda text, label : text)
vectorized_layer.adapt(train_text)

model.add(vectorized_layer)

In [10]:
# creating an embedding layer

model.add(Embedding(max_tokens + 1, 128))
# rnn = RNN(SimpleRNNCell(64), return_sequences = False, return_state = False) # can change it from here
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [11]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [12]:
model.fit(train_data, epochs = 10)

Epoch 1/10
782/782 [==============================] - 108s 133ms/step - loss: 0.5117 - accuracy: 0.7360
Epoch 2/10
782/782 [==============================] - 106s 135ms/step - loss: 0.4214 - accuracy: 0.8045
Epoch 3/10
782/782 [==============================] - 107s 137ms/step - loss: 0.3920 - accuracy: 0.8220
Epoch 4/10
782/782 [==============================] - 108s 139ms/step - loss: 0.3617 - accuracy: 0.8398
Epoch 5/10
782/782 [==============================] - 109s 139ms/step - loss: 0.3290 - accuracy: 0.8563
Epoch 6/10
782/782 [==============================] - 107s 136ms/step - loss: 0.2979 - accuracy: 0.8745
Epoch 7/10
782/782 [==============================] - 109s 139ms/step - loss: 0.2685 - accuracy: 0.8860
Epoch 8/10
782/782 [==============================] - 107s 137ms/step - loss: 0.2410 - accuracy: 0.9002
Epoch 9/10
782/782 [==============================] - 108s 138ms/step - loss: 0.2193 - accuracy: 0.9086
Epoch 10/10
782/782 [==============================] - 107s 137m

In [13]:
model.evaluate(test_data)

782/782 [==============================] - 31s 38ms/step - loss: 0.7303 - accuracy: 0.7685


[0.7302866578102112, 0.7685199975967407]

In [14]:
model.predict(['I loved the movie'])

array([[0.99407613]], dtype=float32)